# Example 2 — Sequence Detector (FSM)

## Origin
This example is taken directly from **ChipChat Example 2** (`sequence_detector`).
In ChipChat, the task was posed as a natural-language question to the LLM:

> *"I am trying to create a Verilog model for a sequence detector. It must meet the following specifications:*
> - *Inputs: Clock, Active-low reset, Data (3 bits)*
> - *Outputs: Sequence found*
>
> *While enabled, it should detect the following sequence of binary input values:*
> `001 → 101 → 110 → 000 → 110 → 110 → 011 → 101`
>
> *How would I write a design that meets these specifications?"*

## Testbench
The testbench `sequence_detector_tb.v` was taken directly from the course assignment
repository (`LLM4ChipDesign/VerilogGenBenchmark/TestBench/`) as provided in the
ChipChat Colab and required by the AutoChip Tutorial Assignment. It is the official course-supplied testbench for this module.

## Why AutoChip instead of ChipChat
AutoChip improves on the ChipChat workflow by:
1. Running **multiple candidate RTL responses per iteration** (3 here) and ranking them
2. Automatically feeding **compiler and simulation errors back** to the LLM as structured feedback
3. Iterating until a candidate achieves **rank 1.0** (testbench prints "All test cases passed.")

## Module specification
- **Module name:** `sequence_detector`
- **Inputs:** `clk`, `reset_n` (active-low synchronous), `data [2:0]`
- **Output:** `sequence_found` (reg) — asserted for exactly 1 cycle when full sequence received
- **Logic type:** Sequential FSM, 8 states S0–S7
- **Sequence:** `001 → 101 → 110 → 000 → 110 → 110 → 011 → 101`
- **Critical constraint:** `sequence_found` must be asserted in the **same clock cycle** the final `101` is received in S7
- **Constraint:** Plain Verilog-2001 only — no `logic`, `typedef`, `enum`, `always_ff`


# Getting set up

In [1]:
#@title Setting up the notebook

### Installing dependencies
!pip install openai tiktoken

!apt-get update
!apt-get install -y iverilog

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://cli.github.com/packages stable InRelease [3,917 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,688 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,717 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [70.9 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe am

In [2]:
import os
# Recommended: Runtime > Secrets > add 'openai_api_key', then uncomment below
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["OPENAI_API_KEY"] = ""  # replace before running

In [3]:
#@title Utility functions

import sys
import os
import openai
import tiktoken
from abc import ABC, abstractmethod
import re
import getopt
import json

################################################################################
### LOGGING
################################################################################
# Allows us to log the output of the model to a file if logging is enabled
class LogStdoutToFile:
    def __init__(self, filename):
        self._filename = filename
        self._original_stdout = sys.stdout

    def __enter__(self):
        if self._filename:
            sys.stdout = open(self._filename, 'w')
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if self._filename:
            sys.stdout.close()
        sys.stdout = self._original_stdout

################################################################################
### CONFIG & ARGS
################################################################################
def load_config(config_file="config.json"):
    """Load and validate the configuration from the specified JSON file."""
    with open(config_file, 'r') as file:
        config = json.load(file)

    if 'general' not in config:
        raise ValueError("Missing general section in config file")

    config_values = config['general']

    # Only parse ensemble settings if specified
    parse_ensemble = config_values.get('ensemble', False)
    ensemble_config = {}
    if parse_ensemble:
        ensemble_config = config.get('ensemble', {})

    #return config_values
    return config_values, ensemble_config


def validate_ensemble_config(ensemble_config, max_iterations):
    seen_start_iterations = set()
    adjusted_config = {}
    has_start_at_zero = False

    for model_name, model_info in ensemble_config.items():
        start_iteration = model_info['start_iteration']

        # Adjust negative start_iteration values
        if start_iteration < 0:
            start_iteration += max_iterations+1

        # Check if start_iteration is within the valid range
        if not (0 <= start_iteration <= max_iterations):
            raise ValueError(f"Invalid start_iteration {model_info['start_iteration']} for {model_name}. "
                             f"Must be within the range of 0 to {max_iterations} or valid negative index.")

        # Check for conflicting start_iterations
        if start_iteration in seen_start_iterations:
            raise ValueError(f"Conflicting start_iteration {start_iteration} for {model_name}. "
                             f"Another model already uses this start iteration.")
        seen_start_iterations.add(start_iteration)

        # Check if there is a model starting at iteration 0
        if start_iteration == 0:
            has_start_at_zero = True

        # Update the adjusted configuration
        adjusted_config[model_name] = {
            "start_iteration": start_iteration,
            "model_family": model_info['model_family'],
            "model_id": model_info['model_id']
        }

        if not has_start_at_zero:
            raise ValueError("No model starting at iteration 0 in the ensemble. One model must start at iteration 0.")

    return adjusted_config


def parse_args_and_config():
    """Parse command-line arguments and merge them with configuration file values."""
    usage = """Usage: auto_create_verilog.py [--help] --prompt=<prompt> --name=<module name> --testbench=<testbench file> --iter=<iterations> --model=<llm family> --model-id=<specific model> --num-candidates=<candidates per request> --outdir=<directory for outputs> --log=<log file>

	-h|--help: Prints this usage message

	-p|--prompt: The initial design prompt for the Verilog module

	-n|--name: The module name, must match the testbench expected module name

	-t|--testbench: The testbench file to be run

	-i|--iter: [Optional] Number of iterations before the tool quits (defaults to 10)

	-m|--model: The LLM family to use. Must be one of the following
		- ChatGPT
		- Claude
		- Mistral
		- Gemini
		- CodeLlama
		- Human (requests user input)

	--model-id: The specific model to use for the model family

	--num-candidates: The number of candidates to rank per tree level

	-o|--outdir: Directory to place all run-specific files in

	-l|--log: [Optional] Log the output of the model to the given file
"""

    config_file = "config.json"

    # Load config values from the file
    config_values, ensemble_config = load_config(config_file)

    required_values = ['prompt', 'name', 'testbench', 'outdir', 'log']
    if not ensemble_config:
        required_values +=['model_family', 'model_id']

    for value in required_values:
        if value not in config_values:
            raise ValueError(f"Missing {value} in general section\n{usage}")


    # general values for optional config values
    if 'num_candidates' not in config_values:
        config_values['num_candidates'] = 1
    if 'iterations' not in config_values:
        config_values['iterations'] = 10


    if ensemble_config:
        ensemble_config = validate_ensemble_config(ensemble_config, config_values['iterations'])

    # Ensure outdir exists
    if config_values['outdir']:
        os.makedirs(config_values['outdir'], exist_ok=True)

    logfile = os.path.join(config_values['outdir'], config_values['log']) if config_values['log'] else None

    #return config_values, logfile
    return config_values, ensemble_config, logfile



################################################################################
### CONVERSATION CLASS
# allows us to abstract away the details of the conversation for use with
# different LLM APIs
################################################################################

class Conversation:
    def __init__(self, log_file=None):
        self.messages = []
        self.log_file = log_file

        if self.log_file and os.path.exists(self.log_file):
            open(self.log_file, 'w').close()

    def add_message(self, role, content):
        """Add a new message to the conversation."""
        self.messages.append({'role': role, 'content': content})

        if self.log_file:
            with open(self.log_file, 'a') as file:
                file.write(f"{role}: {content}\n")

    def get_messages(self):
        """Retrieve the entire conversation."""
        return self.messages

    def get_last_n_messages(self, n):
        """Retrieve the last n messages from the conversation."""
        return self.messages[-n:]

    def remove_message(self, index):
        """Remove a specific message from the conversation by index."""
        if index < len(self.messages):
            del self.messages[index]

    def get_message(self):
        """Retrieve a specific message from the conversation by index."""
        return self.messages[index] if index < len(self.messages) else None

    def clear_messages(self):
        """Clear all messages from the conversation."""
        self.messages = []

    def __str__(self):
        """Return the conversation in a string format."""
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.messages])

################################################################################
### LLM CLASSES
# Defines an interface for using different LLMs so we can easily swap them out
################################################################################
class AbstractLLM(ABC):
    """Abstract Large Language Model."""

    def __init__(self):
        pass

    @abstractmethod
    def generate(self, conversation: Conversation, num_candidates=1):
        """Generate a response based on the given conversation."""
        pass


class ChatGPT(AbstractLLM):
    """ChatGPT Large Language Model."""

    def __init__(self, model_id="gpt-4o"):
        super().__init__()
        openai.api_key=os.environ['OPENAI_API_KEY']
        self.client = openai.OpenAI()
        self.model_id = model_id

    def generate(self, conversation: Conversation, num_candidates=1):
        messages = [{"role" : msg["role"], "content" : msg["content"]} for msg in conversation.get_messages()]


        #print(f"model_id: {self.model_id}")
        #print(f"messages: {messages}")
        #print(f"num_candidates: {num_candidates}")

        response = self.client.chat.completions.create(
        model=self.model_id,
        n=num_candidates,
        messages=messages,
        temperature=0.1
)


        return [c.message.content for c in response.choices]

class LLMResponse():
    """Class to store the response from the LLM"""
    def __init__(self, iteration, response_num, full_text):
        self.iteration = iteration
        self.response_num = response_num

        self.full_text = full_text
        self.tokens = 0

        self.parsed_text = ""
        self.parsed_length = 0

        self.feedback = ""
        self.compiled = False
        self.rank = -3
        self.message = ""

    def set_parsed_text(self, parsed_text):
        self.parsed_text = parsed_text
        self.parsed_length = len(parsed_text)

    def parse_verilog(self):
        module_list = find_verilog_modules(self.full_text)
        if not module_list:
            print("No modules found in response")
            self.parsed_text = ""
        else:
            for module in module_list:
                self.parsed_text += module + "\n\n"
        self.parsed_length = len(self.parsed_text)

    def calculate_rank(self, outdir, module, testbench):
        filename = os.path.join(outdir, module + ".sv")
        vvp_file = os.path.join(outdir, module + ".vvp")

        compiler_cmd = f"iverilog -Wall -Winfloop -Wno-timescale -g2012 -s tb_sequence_detector -o {vvp_file} {filename} {testbench}"
        simulator_cmd = f"vvp -n {vvp_file}"

        try:
            comp_return, comp_err, comp_out = compile_iverilog(outdir, module, compiler_cmd, self)
        except ValueError as e:
            print(e)
            self.rank = -2
            return

        # -----------------------------
        # Compilation handling
        # -----------------------------
        if comp_return != 0:
            self.feedback = comp_err
            self.compiled = False
            print("Compilation error")
            print(comp_err)
            self.message = (
                "The design failed to compile. Please fix the module. "
                "The output of iverilog is as follows:\n" + comp_err
            )
            self.rank = -1
            return

        elif comp_err != "":
            self.feedback = comp_err
            self.compiled = True
            print("Compilation warning")
            print(comp_err)
            self.message = (
                "The design compiled with warnings. Please fix the module. "
                "The output of iverilog is as follows:\n" + comp_err
            )
            self.rank = -0.5
            return

        # -----------------------------
        # Simulation handling
        # -----------------------------
        sim_return, sim_err, sim_out = simulate_iverilog(simulator_cmd)

        print("Simulation output:")
        print(sim_out)

        # Success case (ChipChat testbench style)
        if "All test cases passed." in sim_out:
            self.compiled = True
            self.feedback = ""
            self.message = "The testbench completed successfully."
            self.rank = 1.0
            print("Testbench ran successfully")
            return

        # Failure case
        else:
            self.compiled = True
            self.feedback = sim_out
            self.message = (
                "The testbench simulated, but had errors. "
                "Please fix the module. Output:\n" + sim_out
            )
            self.rank = 0.0
            print("Simulation failed")
            return


################################################################################
### PARSING AND TEXT MANIPULATION FUNCTIONS
################################################################################
# Define the cost per million tokens
COST_PER_MILLION_INPUT_TOKENS_GPT4 = 5.0
COST_PER_MILLION_OUTPUT_TOKENS_GPT4 = 15.0

COST_PER_MILLION_INPUT_TOKENS_GPT4M = 0.15
COST_PER_MILLION_OUTPUT_TOKENS_GPT4M = 0.60

COST_PER_MILLION_INPUT_TOKENS_GPT = 0.50
COST_PER_MILLION_OUTPUT_TOKENS_GPT = 1.50

COST_PER_MILLION_INPUT_TOKENS_CLAUDE = 0.25
COST_PER_MILLION_OUTPUT_TOKENS_CLAUDE = 1.25

# Function to count tokens
def count_tokens(model_family, text):
    #print(f"Counting tokens for string: {text}")
    if model_family == "GPT" or model_family == "GPT4" or model_family == "GPT4M":
        return len(tiktoken.get_encoding("cl100k_base").encode(text))
    elif model_family == "claude":
        return anthropic.Client().count_tokens(text)
    else:
        raise ValueError(f"Unsupported model family: {model_family}")


def calculate_cost(model_family,input_strings,output_strings):
    input_tokens = sum(count_tokens(model_family, text) for text in input_strings)
    output_tokens = sum(count_tokens(model_family, text) for text in output_strings)
    if model_family == "GPT":
        cost_input = (input_tokens / 1_000_000) * COST_PER_MILLION_INPUT_TOKENS_GPT
        cost_output = (output_tokens / 1_000_000) * COST_PER_MILLION_OUTPUT_TOKENS_GPT
    elif model_family == "GPT4":
        cost_input = (input_tokens / 1_000_000) * COST_PER_MILLION_INPUT_TOKENS_GPT4
        cost_output = (output_tokens / 1_000_000) * COST_PER_MILLION_OUTPUT_TOKENS_GPT4
    elif model_family == "GPT4M":
        cost_input = (input_tokens / 1_000_000) * COST_PER_MILLION_INPUT_TOKENS_GPT4M
        cost_output = (output_tokens / 1_000_000) * COST_PER_MILLION_OUTPUT_TOKENS_GPT4M
    elif model_family == "claude":
        cost_input = (input_tokens / 1_000_000) * COST_PER_MILLION_INPUT_TOKENS_CLAUDE
        cost_output = (output_tokens / 1_000_000) * COST_PER_MILLION_OUTPUT_TOKENS_CLAUDE
    else:
        raise ValueError(f"Unsupported model family: {model_family}")
    total_cost = cost_input + cost_output
    return total_cost, input_tokens, output_tokens


def format_message(role, content):
    return f"\n{{role : '{role}', content : '{content}'}}"

def find_verilog_modules(markdown_string):
    """Find all Verilog modules in the markdown string"""
    # Regular expression to match module definitions with or without parameters
    module_pattern = r'\bmodule\b\s+[\w\\_]+\s*(?:#\s*\([^)]*\))?\s*\([^)]*\)\s*;.*?endmodule\b'
    # Find all matches in the input string
    matches = re.findall(module_pattern, markdown_string, re.DOTALL)
    # Process matches to replace escaped characters
    processed_matches = [match.replace('\\_', '_') for match in matches]
    return processed_matches

def write_code_blocks_to_file(markdown_string, module_name, filename):
    # Find all code blocks using a regular expression (matches content between triple backticks)
    code_match = find_verilog_modules(markdown_string)

    if not code_match:
        print("No code blocks found in response")
        exit(3)

    # Open the specified file to write the code blocks
    with open(filename, 'w') as file:
        for code_block in code_match:
            file.write(code_block)
            file.write('\n')


def generate_verilog(conv, model_type, model_id=""):
    if model_type == "ChatGPT":
        model = ChatGPT()
    else:
        raise ValueError("Invalid model type")
    return(model.generate(conv))

def compile_iverilog(outdir,module,compiler_cmd,response:LLMResponse):
    """Compile the Verilog module and return the output"""

    filename = os.path.join(outdir,module+".sv")
    write_code_blocks_to_file(response.parsed_text, "module", filename)

    attempt = 0
    while attempt < 3:
        try:
            proc = subprocess.run(compiler_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=120)
            return proc.returncode, proc.stderr, proc.stdout
        except subprocess.TimeoutExpired:
            attempt += 1
            if attempt >= 3:
                raise ValueError("Compilation attempts timed out")

def simulate_iverilog(simulation_cmd):
    """Compile the Verilog module and return the output"""

    attempt = 0
    while attempt < 3:
        try:
            proc = subprocess.run(simulation_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=120)
            return proc.returncode, proc.stderr, proc.stdout
        except subprocess.TimeoutExpired:
            attempt += 1
            if attempt >= 3:
                raise ValueError("Simulation attempts timed out")

def generate_verilog_responses(conv, model_type, model_id="", num_candidates=1):
    match model_type:
        case "ChatGPT":
            model = ChatGPT(model_id)
        case _:
            raise ValueError("Invalid model type")

    return(model.generate(conversation=conv, num_candidates=num_candidates))

def get_iteration_ensemble(iteration, ensemble_config):

    sorted_ensemble = sorted(ensemble_config.values(), key=lambda x: x['start_iteration'], reverse=True)

    family = None
    model_id = None
    for ensemble_info in sorted_ensemble:
        if iteration >= ensemble_info['start_iteration']:
            family = ensemble_info['model_family']
            model_id = ensemble_info['model_id']
            break
    return family, model_id


In [4]:
import subprocess
import sys
import os

def verilog_loop(design_prompt, module, testbench, max_iterations, model_type, model_id="", num_candidates=5, outdir="", log=None, ensemble_config={}):

    if outdir != "":
        outdir = outdir + "/"

    conv = Conversation(log_file=log)

    #conv.add_message("system", "You are a Verilog engineering tool. Given a design specification you will provide a Verilog module in response. Given errors in that design you will provide a completed fixed module. Only complete functional models should be given. No testbenches should be written under any circumstances, as those are to be written by the human user.")
    conv.add_message("system", """
    You are an expert Verilog RTL engineer.

    Strict rules:
    - Use plain Verilog-2001 only.
    - Do NOT use typedef.
    - Do NOT use enum.
    - Do NOT use logic type.
    - Use reg and wire only.
    - Use parameter for state encoding.
    - Use posedge clk.
    - Active-low synchronous reset.
    - Fully synthesizable.

    You must implement an 8-state deterministic FSM.

    Detect exactly this sequence:

    001 → 101 → 110 → 000 → 110 → 110 → 011 → 101

    CRITICAL:
    The output sequence_found must be asserted
    in the SAME clock cycle that the final 3'b101 is received.

    It must NOT be delayed to the next clock cycle.

    Return ONLY one complete Verilog module inside triple backticks.
    """)





    #with open(testbench, 'r') as file: testbench_text = file.read()
    #full_prompt = design_prompt + "\n\nThe module will be tested with the following testbench:\n\n" + testbench_text + "\n\n"

    with open(testbench, 'r') as file:
      testbench_text = file.read()

    full_prompt = design_prompt + "\n\nThe module will be tested with the following testbench:\n\n" + testbench_text

    conv.add_message("user", full_prompt)


    success = False
    timeout = False

    iterations = 0

    global_max_response = LLMResponse(-3,-3,"")


    ##############################

    while not (success or timeout):


        if ensemble_config:
            print(f"Getting model from ensemble")
            model_type, model_id = get_iteration_ensemble(iterations, ensemble_config)

        print(f"Iteration: {iterations}")
        print(f"Model type: {model_type}")
        print(f"Model ID: {model_id}")
        print(f"Number of responses: {num_candidates}")

        response_texts=generate_verilog_responses(conv, model_type, model_id, num_candidates=num_candidates)

        responses = [LLMResponse(iterations,response_num,response_text) for response_num,response_text in enumerate(response_texts)]
        for index, response in enumerate(responses):

            response_outdir = os.path.join(outdir, f"iter{str(iterations)}/response{index}/")
            if not os.path.exists(response_outdir):
                os.makedirs(response_outdir)


            response_cost = 0
            input_tokens = 0
            output_tokens = 0

            response.parse_verilog()
            if response.parsed_text == "":
                response.rank = -2
                response.message = "No modules found in response"
            else:
                response.calculate_rank(response_outdir, module, testbench)

            input_messages = [msg['content'] for msg in conv.get_messages() if msg['role'] == 'user' or msg['role'] == 'system']
            output_messages = [msg['content'] for msg in conv.get_messages() if msg['role'] == 'assistant']
            output_messages.append(response.parsed_text)
            if model_type == "ChatGPT" and model_id == "gpt-4o":
                response_cost, input_tokens, output_tokens = calculate_cost("GPT4",input_messages,output_messages)
            elif model_type == "ChatGPT" and model_id == "gpt-4o-mini":
                response_cost, input_tokens, output_tokens = calculate_cost("GPT4M",input_messages,output_messages)
            elif model_type == "ChatGPT" and model_id == "gpt-3.5-turbo":
                response_cost, input_tokens, output_tokens = calculate_cost("GPT",input_messages,output_messages)
            elif model_type == "Claude":
                response_cost, input_tokens, output_tokens = calculate_cost("claude",input_messages,output_messages)


            print(f"Cost for response {index}: ${response_cost:.10f}")

            with open(os.path.join(response_outdir,f"log.txt"), 'w') as file:
                file.write('\n'.join(str(i) for i in conv.get_messages()))
                file.write(format_message("assistant", response.full_text))
                file.write('\n\n Iteration rank: ' + str(response.rank) + '\n') ## FIX

                file.write(f"\n Model: {model_id}")
                file.write(f"\n Input tokens: {input_tokens}")
                file.write(f"\n Output tokens: {output_tokens}")
                file.write(f"\nTotal cost: ${response_cost:.10f}\n")

        ## RANK RESPONSES
        max_rank_response = max(responses, key=lambda resp: (resp.rank, -resp.parsed_length))
        if max_rank_response.rank > global_max_response.rank:
            global_max_response = max_rank_response
        elif max_rank_response.rank == global_max_response.rank and max_rank_response.parsed_length > global_max_response.parsed_length:
            global_max_response = max_rank_response

        print(f"Response ranks: {[resp.rank for resp in responses]}")
        print(f"Response lengths: {[resp.parsed_length for resp in responses]}")

        conv.add_message("assistant", max_rank_response.parsed_text)

        if max_rank_response.rank == 1:
            success = True



################################


        if not success:
            if iterations > 0:
                conv.remove_message(2)
                conv.remove_message(2)

            #with open(testbench, 'r') as file: testbench_text = file.read()
            #message = message + "\n\nThe testbench used for these results is as follows:\n\n" + testbench_text
            #message = message + "\n\nCommon sources of errors are as follows:\n\t- Use of SystemVerilog syntax which is not valid with iverilog\n\t- The reset must be made asynchronous active-low\n"
            feedback_message = f"""
            The previous FSM failed simulation.

            Simulation error:
            {max_rank_response.feedback}

            The FSM reached the final state but did NOT assert
            sequence_found in the same clock cycle as the final 3'b101 input.

            Fix the output logic so that:
            - sequence_found is asserted in the SAME cycle
              that data == 3'b101 in state S7.
            - It must NOT be delayed to the next cycle.
            - It must be deasserted otherwise.

            Return the full corrected module.
            """


            conv.add_message("user", feedback_message)


        if iterations >= max_iterations:
            timeout = True

        iterations += 1

    return global_max_response



In [5]:
config = {
  "general": {
    "name": "sequence_detector",
    "prompt": "prompt.txt",
    "testbench": "sequence_detector_tb.v",
    "iterations": 6,
    "num_candidates": 3,
    "outdir": "out",
    "log": "trajectory.log",
    "model_family": "ChatGPT",
    "model_id": "gpt-4o"
  }
}


In [6]:
import json
with open("config.json", "w") as f:
    json.dump(config, f, indent=2)

print(json.dumps(config, indent=2))
print("config.json written successfully.")

{
  "general": {
    "name": "sequence_detector",
    "prompt": "prompt.txt",
    "testbench": "sequence_detector_tb.v",
    "iterations": 6,
    "num_candidates": 3,
    "outdir": "out",
    "log": "trajectory.log",
    "model_family": "ChatGPT",
    "model_id": "gpt-4o"
  }
}
config.json written successfully.


In [7]:
design_prompt = """
Implement module sequence_detector.

Interface:

module sequence_detector(
    input  wire clk,
    input  wire reset_n,
    input  wire [2:0] data,
    output reg sequence_found
);

Design an 8-state FSM (S0–S7).

Transitions:
S0: expect 001
S1: expect 101
S2: expect 110
S3: expect 000
S4: expect 110
S5: expect 110
S6: expect 011
S7: expect 101

Only when the final 101 is received in S7,
sequence_found must be 1 for one cycle.

All other cycles must output 0.
Return to S0 on mismatch.
"""


# Write prompt to file (required by config)
with open("prompt.txt", "w") as f:
    f.write(design_prompt)
print("prompt.txt written.")

prompt.txt written.


## Testbench — `sequence_detector_tb.v`

The testbench `sequence_detector_tb.v` is the official course-supplied benchmark
from `LLM4ChipDesign/VerilogGenBenchmark/TestBench/`. It drives the module through
the full 8-step sequence across 8 clock cycles:

| Cycle | Input `data` | Expected `sequence_found` |
|-------|-------------|--------------------------|
| 1 | `001` | `0` |
| 2 | `101` | `0` |
| 3 | `110` | `0` |
| 4 | `000` | `0` |
| 5 | `110` | `0` |
| 6 | `110` | `0` |
| 7 | `011` | `0` |
| 8 | `101` | `1` ← must assert same cycle |

It also checks **mismatch recovery** — a wrong input mid-sequence must return the FSM to S0.
A pass prints `"All test cases passed."` The top-level module is `tb_sequence_detector`.


In [8]:
!curl -L -o sequence_detector_tb.v \
https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/main/VerilogGenBenchmark/TestBench/sequence_detector_tb.v

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2114  100  2114    0     0  14877      0 --:--:-- --:--:-- --:--:-- 14992


In [9]:
with open("sequence_detector_tb.v") as f:
    print(f.read())

`timescale 1ns/1ps

module tb_sequence_detector();
    reg clk;
    reg reset_n;
    reg [2:0] data;
    wire sequence_found;

    // Instantiate the sequence_detector module
    sequence_detector dut (
        .clk(clk),
        .reset_n(reset_n),
        .data(data),
        .sequence_found(sequence_found)
    );

    // Clock generation
    always begin
        #5 clk = ~clk;
    end

    // Test stimulus task
    task apply_stimulus;
        input [2:0] data_value;
        input integer delay_cycles;
        begin
            data <= data_value;
            repeat (delay_cycles) @(posedge clk);
        end
    endtask

    // Check output task
    task check_output;
        input integer cycle;
        input expected_value;
        begin
            if (sequence_found !== expected_value) begin
                $display("Error: Cycle %0d, Expected: %b, Got: %b", cycle, expected_value, sequence_found);
                $finish;
            end
        end
    endtask

    // Testbench 

## AutoChip Trajectory

**Model:** `gpt-4o` | **Max iterations:** 6 | **Candidates per iteration:** 3 | **Temperature:** default

**Key design challenge:** The repeated `110` inputs at positions 5 and 6 require **two distinct
states** (S4 and S5). A single state with a self-loop on `110` loses count of how many
consecutive `110`s have been seen, causing `sequence_found` to never assert on the final `101`.

**Iteration 0:** All 3 candidates failed simulation.
Output: `Error: Cycle 8, Expected: 1, Got: 0` — `sequence_found` was never asserted.
Likely cause: LLM merged S4/S5 into a single state or used delayed output logic.
Feedback sent: *"The testbench simulated, but had errors."*

**Iteration 1:** All 3 candidates passed immediately.
`Response ranks: [1.0, 1.0, 1.0]` — `All test cases passed.`
LLM correctly implemented separate S4 and S5 states with combinational
output asserting `sequence_found` in the same cycle as the final `101`.

AutoChip terminated after **iteration 1** with `Final Best Rank: 1.0`.


In [10]:
config_values, ensemble_config, logfile = parse_args_and_config()

result = verilog_loop(
    design_prompt=design_prompt,
    module=config_values["name"],
    testbench=config_values["testbench"],
    max_iterations=config_values["iterations"],
    model_type=config_values["model_family"],
    model_id=config_values["model_id"],
    num_candidates=config_values["num_candidates"],
    outdir=config_values["outdir"],
    log=logfile,
    ensemble_config=ensemble_config
)

print("Final Best Rank:", result.rank)

Iteration: 0
Model type: ChatGPT
Model ID: gpt-4o
Number of responses: 3
Simulation output:
Error: Cycle 8, Expected: 1, Got: 0

Simulation failed
Cost for response 0: $0.0114500000
Simulation output:
Error: Cycle 8, Expected: 1, Got: 0

Simulation failed
Cost for response 1: $0.0116600000
Simulation output:
Error: Cycle 8, Expected: 1, Got: 0

Simulation failed
Cost for response 2: $0.0114500000
Response ranks: [0.0, 0.0, 0.0]
Response lengths: [1436, 1453, 1436]
Iteration: 1
Model type: ChatGPT
Model ID: gpt-4o
Number of responses: 3
Simulation output:
All test cases passed.

Testbench ran successfully
Cost for response 0: $0.0183500000
Simulation output:
All test cases passed.

Testbench ran successfully
Cost for response 1: $0.0183500000
Simulation output:
All test cases passed.

Testbench ran successfully
Cost for response 2: $0.0183500000
Response ranks: [1.0, 1.0, 1.0]
Response lengths: [1331, 1331, 1331]
Final Best Rank: 1.0


In [11]:
# Explicit compile and simulate commands (mirrors AutoChip internal calls)
!iverilog -Wall -Winfloop -Wno-timescale -g2012 \
    -s tb_sequence_detector \
    -o out/sequence_detector_final.vvp \
    out/iter1/response0/sequence_detector.sv \
    sequence_detector_tb.v

!vvp -n out/sequence_detector_final.vvp

All test cases passed.


## Part I(b) — Manual RTL Design

**Design choices:**
- **`parameter` state encoding** — 8 states S0–S7 as plain integers. No `typedef`/`enum`
  to stay strict Verilog-2001 compliant with iverilog.
- **S4 and S5 are separate states** — both expect `110` but must be counted independently.
  A single state with a self-loop cannot distinguish "first `110`" from "second `110`",
  which breaks the sequence count at positions 5 and 6.
- **Synchronous active-low reset** — `if (!reset_n)` inside `always @(posedge clk)`
  resets state to S0 on the clock edge, matching the testbench expectation.
- **Combinational output** — `sequence_found` is driven by an `always @(*)` block that
  checks `state == S7 && data == 3'b101`, satisfying the same-cycle assertion requirement.
- **Mismatch → S0** — any unexpected input in any state returns the FSM to S0,
  matching the testbench recovery checks.


In [12]:
manual_rtl = """
// Manual RTL — sequence_detector_manual.v
// Part I(b): Hand-written Verilog-2001, 8-state sequence detector FSM

module sequence_detector (
    input  wire       clk,
    input  wire       reset_n,
    input  wire [2:0] data,
    output reg        sequence_found
);

    // State encoding — plain parameters, Verilog-2001 compliant
    parameter S0 = 3'd0, S1 = 3'd1, S2 = 3'd2, S3 = 3'd3;
    parameter S4 = 3'd4, S5 = 3'd5, S6 = 3'd6, S7 = 3'd7;

    reg [2:0] state, next_state;

    // State register — synchronous active-low reset
    always @(posedge clk) begin
        if (!reset_n)
            state <= S0;
        else
            state <= next_state;
    end

    // Next-state logic
    always @(*) begin
        case (state)
            S0: next_state = (data == 3'b001) ? S1 : S0;
            S1: next_state = (data == 3'b101) ? S2 : S0;
            S2: next_state = (data == 3'b110) ? S3 : S0;
            S3: next_state = (data == 3'b000) ? S4 : S0;
            S4: next_state = (data == 3'b110) ? S5 : S0;
            S5: next_state = (data == 3'b110) ? S6 : S0;
            S6: next_state = (data == 3'b011) ? S7 : S0;
            S7: next_state = S0;
            default: next_state = S0;
        endcase
    end

    // Output logic — combinational, same-cycle assertion
    always @(*) begin
        sequence_found = (state == S7 && data == 3'b101) ? 1'b1 : 1'b0;
    end

endmodule
"""

with open("sequence_detector_manual.v", "w") as f:
    f.write(manual_rtl)
print(manual_rtl)


// Manual RTL — sequence_detector_manual.v
// Part I(b): Hand-written Verilog-2001, 8-state sequence detector FSM

module sequence_detector (
    input  wire       clk,
    input  wire       reset_n,
    input  wire [2:0] data,
    output reg        sequence_found
);

    // State encoding — plain parameters, Verilog-2001 compliant
    parameter S0 = 3'd0, S1 = 3'd1, S2 = 3'd2, S3 = 3'd3;
    parameter S4 = 3'd4, S5 = 3'd5, S6 = 3'd6, S7 = 3'd7;

    reg [2:0] state, next_state;

    // State register — synchronous active-low reset
    always @(posedge clk) begin
        if (!reset_n)
            state <= S0;
        else
            state <= next_state;
    end

    // Next-state logic
    always @(*) begin
        case (state)
            S0: next_state = (data == 3'b001) ? S1 : S0;
            S1: next_state = (data == 3'b101) ? S2 : S0;
            S2: next_state = (data == 3'b110) ? S3 : S0;
            S3: next_state = (data == 3'b000) ? S4 : S0;
            S4: next_state = (da

In [13]:
# Verify manual RTL with the same course-supplied testbench
!iverilog -Wall -Winfloop -Wno-timescale -g2012 \
    -s tb_sequence_detector \
    -o out/manual.vvp \
    sequence_detector_manual.v \
    sequence_detector_tb.v

!vvp -n out/manual.vvp

All test cases passed.
